# Ensemble Features - KPCA-MNL

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from model.resnet import resnet18
from model.inception import inception_v3
from model.vgg import vgg16
from model.alexnet import alexnet
from data_loader.data_loader import get_data_loader
from model.utils import LayerActivations
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
model1 = vgg16(pretrained=False, num_classes=2)
model2 = resnet18(pretrained=False, num_classes=2)
model3 = alexnet(pretrained=False, num_classes=2)

In [4]:
# load checkpoint
model1 = model1.to(device)
best_vgg = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_vgg.pth')
model1.load_state_dict(best_vgg)
model1.train(False)
model1.eval()

model2 = model2.to(device)
best_resnet = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_resnet.pth')
model2.load_state_dict(best_resnet)
model2.train(False)
model2.eval()

model3 = model3.to(device)
best_alex = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_alex.pth')
model3.load_state_dict(best_alex)
model3.train(False)
model3.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [5]:
model1 = nn.Sequential(*list(model1.children())[:-2])

In [6]:
model2 = nn.Sequential(*list(model2.children())[:-2])

In [7]:
model3 = nn.Sequential(*list(model3.children())[:-1])

In [8]:
data_loader = get_data_loader()

In [22]:
# Extract features
# trn_labels = []
# trn_features = []
final_df = pd.DataFrame()
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out_1 = model1(d)
        out_2 = model2(d)
        out_3 = model3(d)
    out_1 = out_1.view(out_1.size(0), -1)
    out_2 = out_2.view(out_2.size(0), -1)
    out_3 = out_3.view(out_3.size(0), -1)
    out = out_1 + out_2 + out_3
    tmp_array = out.cpu().data.numpy()
    tmp_df = pd.DataFrame(tmp_array)
    la_lst = la
    tmp_df['y'] = la_lst
    final_df = final_df.append(tmp_df, ignore_index=True)
    

RuntimeError: The size of tensor a (25088) must match the size of tensor b (9216) at non-singleton dimension 1

In [19]:
tmp2 = out_1+out_2

In [21]:
tmp2

tensor([[0.8305, 0.3085, 1.0267,  ..., 0.0130, 0.5532, 0.4117],
        [0.3196, 0.0000, 0.0000,  ..., 0.4060, 1.1837, 0.0158],
        [0.0406, 2.0106, 2.2321,  ..., 0.0000, 0.0000, 0.3842]],
       device='cuda:0')

In [15]:
out_3.shape

torch.Size([3, 9216])

In [18]:
d.shape

torch.Size([3, 3, 224, 224])

In [17]:
model2

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [ ]:
# Extract features for VGG16
trn_labels = []
trn_vgg_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model1(d)
    out = out.view(out.size(0), -1)
    trn_labels.extend(la)
    trn_vgg_features.extend(out.cpu().data.numpy().tolist())

val_labels = []
val_vgg_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model1(d)
    out = out.view(out.size(0), -1)
    val_labels.extend(la)
    val_vgg_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
# Extract features for ResNet
trn_resnet_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model2(d)
    out = out.view(out.size(0), -1)
    trn_resnet_features.extend(out.cpu().data.numpy().tolist())

val_resnet_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model2(d)
    out = out.view(out.size(0), -1)
    val_resnet_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
# Extract features for Alexnet
trn_alex_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model3(d)
    out = out.view(out.size(0), -1)
    trn_alex_features.extend(out.cpu().data.numpy().tolist())

val_alex_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model3(d)
    out = out.view(out.size(0), -1)
    val_alex_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
class FeaturesDataset(Dataset):
    def __init__(self,featlst1,featlst2,featlst3,labellst):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.featlst3 = featlst3
        self.labellst = labellst

    def __getitem__(self,index):
        return (self.featlst1[index]+self.featlst2[index]+self.featlst3[index],self.labellst[index])

    def __len__(self):
        return len(self.labellst)

In [ ]:
trn_ensm_dset = FeaturesDataset(trn_vgg_features, trn_resnet_features, trn_alex_features, trn_labels)
val_ensm_dset = FeaturesDataset(val_vgg_features, val_resnet_features, val_alex_features, val_labels)

trn_ensm_loader = DataLoader(trn_ensm_dset,batch_size=8,shuffle=True)
val_ensm_loader = DataLoader(val_ensm_dset,batch_size=8)

In [ ]:
# Extract features
def tensor_to_pd(data_loader):
    final_df = pd.DataFrame()
    for inputs, labels in tqdm(data_loader):
        tmp_df = pd.DataFrame(inputs)
        tmp_df['y'] = 

In [ ]:
da, la = next(iter(trn_ensm_loader))

In [ ]:
tmp_df = pd.DataFrame(np.array(da))

In [ ]:
trn_ensm_loader.